In [3]:
import sympy as sp
def Mod(a, b):
    return a%b

def mod_binary(g, k, n, p):
    y=1
    k = str(bin(k))[2:]
    for i in k:
        if i=='1':
            y = Mod(Mod(y**2, p)*g, p)
            # print(y)
        else:
            y = Mod(y**2, p)
            # print(y)
    return y

def inv(a, n):
    # return a^-1 mod n
    return mod_binary(a, n-2, len(str(bin(n-2))[2:]) , n)

def pollard_rho(g, y, p, q):
    # q = (p-1) // 2
    # ランダムに移動するための関数
    def new_xab(x, a, b,  g, y, p, q):
        subset = x % 3
        if subset == 0:
            return ((x*x) % p, (a*2) % q, (b*2) % q)
        if subset == 1:
            return ((x*g) % p, (a+1) % q, b        )
        if subset == 2:
            return ((x*y) % p, a        , (b+1) % q)
    # フロイドの循環検出法
    x, a, b = 1, 0, 0
    X, A, B = x, a, b
    for i in range(1, p):
        x, a, b = new_xab(x, a, b,  g, y, p, q)
        X, A, B = new_xab(X, A, B,  g, y, p, q)
        X, A, B = new_xab(X, A, B,  g, y, p, q)
        if x == X:
            break
    # res = ((a - A) * pow(B - b, -1, q)) % q
    res = ((a-A) * inv(B-b, q)) % q
    if pow(g, res, p) == y:
    # if mod_binary(g, res, len(str(bin(res))[2:]), p) == y:
        return res
    if pow(g, res + q, p) == y:
    # if mod_binary(g, res+q, len(str(bin(res+q))[2:]), p) == y:
        return res + q
    return None

In [32]:
def rho_method(g, h, p, l):
    def f(x, g, h, p, a, b):
        if x % 3 == 0:
            a = (a+1) % p
            return ((g*x) % p, a, b)
        elif x % 3 == 1:
            b = (b+1) % p
            return ((h*x) % p, a, b)
        else:
            a, b = (a*2) % p, (b*2) % p
            return ((x*x) % p, a, b)
    a, b = 1, 0
    x_lst = [(g**a * h**b % p, a, b)]
    x = x_lst[-1][0]
    d = 100
    while x not in x_lst:
        x, a, b = f(x, g, h, p, a, b)
        if x % d == 0:
            if x in x_lst:
                collision1, collision2 =  x_lst.index(x), (x, a, b)
                break
            else:
                x_lst.append((x, a, b))
    
    a1, b1 = collision1[1], collision1[2]
    a2, b2 = collision2[1], collision2[2]
    # print(a1, b1, a2, b2)
    return ((a1 - a2) * inv(b2 - b1, l)) % l
        
        

In [1]:
import sys
def rho_method2(g, h, p, l, a0, b0, d):
    def random_walk(g, h, p, l, a, b, x):
        if x % 3 ==0:
            a = (a+1) % l
            x = (x*g) % p
        elif x % 3 == 1:
            b = (b+1) % l
            x = (x*h) % p
        else:
            a = (a*2) % l
            b = (b*2) % l
            x = (x*x) % p
        return [a, b, x]
    
    def calculate_dl(g, h, p, l, a, b):
        a = l-a
        x, y, d = sp.gcdex(b, l)
        if a % d != 0:
            sys.stderr.write("can not solve")
            return None
        a, b, l = a//d, b//d, l//d
        x0 = x * a % l
        for _ in range(d):
            if mod_binary(g, x0, len(str(bin(x0))[2:]), p) == h:
                return x0
            x0 += l
        return None

    cnt = 0
    dict_x = dict()
    x0 = (mod_binary(g, a0, len(str(bin(p))[2:]), p) * mod_binary(h, b0, len(str(bin(p))[2:]), p)) % p
    dict_x[x0] = a0, b0
    while True:
        [a0, b0, x0] = random_walk(g, h, p, l, a0, b0, x0)
        cnt += 1
        if dict_x.get(x0) != None:
            ap, bp = dict_x[x0]
            a = (a0 - ap) % l
            b = (b0 - bp) % l
            x = calculate_dl(g, h, p, l, a, b)
            if x == None:
                return None
            else:
                return [x, cnt, dict_x.__sizeof__()]
        else:
            dict_x[x0] = a0, b0

        if cnt % d == 0:
            dict_x[x0] = a0, b0

In [ ]:
# 問4.1
import math
import random
import time

def Inverse(g, p):
  # 拡張ユークリッドの互除法
  a = []
  x = []
  y = []
  q = []
  a.extend([g, p])
  i = 1
  q.append(0)
  x.extend([1, 0])
  y.extend([0, 1])

  while 1:
    if a[i] == 0:
      break
    a.append(a[i-1] % a[i])
    q.append(a[i-1] // a[i])
    x.append(x[i-1] - q[i] * x[i])
    y.append(y[i-1] - q[i] * y[i])
    i += 1
  
  if x[i-1] < 0:
    x[i-1] += p

  return [a[i-1], x[i-1], y[i-1]]

def ModBinary1(k, g, p):
  n = []
  i = k
  while i > 1:
    n.append(i % 2)
    i = i // 2
  n.append(i)

  y = 1
  i = len(n)
  while i > 0:
    if n[i-1] % 2 == 1:
      y = pow(y, 2, p)*g % p
    else:
      y = pow(y, 2, p)
    i -= 1
  return y

# 課題5
p = 1077984309859658267861
l = 8693421853706921515
g = 657139733149567012977
h = 772772310495087727220
p = 554860571
l = 5044187
g = 397628281
h = 480992339
h = 404852643

# a0 = random.randint(0, l-1)
# b0 = random.randint(0, l-1)
a0 = 1
b0 = 0
print("a0:{}\nb0:{}\n".format(a0, b0))

def rho(g, h, p, l, a0, b0):
  ts = time.time()
  a = a0
  b = b0
  x = {}
  i = 0
  x0 = (ModBinary1(a, g, p) * ModBinary1(b, h, p)) % p

  # distinguish pointの設定
  # 課題1 1000000       0.00200152s, 6096times
  # 課題2 1000000000        0.0220s, 39341times
  # 課題3 10000000000       11.265s, 20516595times
  # 課題4 10000000000000    265.05s, 465325067times
  if p > 100000000000000000:
    d = (p-1)//100000000000000000
  else:
    d = 1
 
  while x0 not in x:
    if i % d == 0:
      x[x0] = a, b
    if x0 % 3 == 1:
      b = (b+1) % l
      x0 = x0 * h % p
    elif x0 % 3 == 2:
      a = (a*2) % l
      b = (b*2) % l
      x0 = pow(x0, 2, p)
    else:
      a = (a+1) % l
      x0 = x0 * g % p
    i += 1

  # xの計算
  x_h = (a-x[x0][0]) * Inverse(x[x0][1]-b, l)[1] % l

  # 期待値
  exp = math.sqrt(l)

  te = time.time()
  return x_h, i-1, exp, te-ts

ans = rho(g, h, p, l, a0, b0)
print("x(h = g^x (mod p)):{}\nrepeat times:{}\ntime:{}".format(ans[0], ans[1], ans[3]))
print(pow(g, ans[0], p)==h) # g^x = h ならTrue、そうでなければFalse

In [4]:
import sys
# g = 397628281
# h = 404852643
# p = 554860571
# l = 5044187

# d = 100
# a0, b0 = 1, 0
# x = rho_method2(g, h, p, l, a0, b0, d)
# print(x)
# print(mod_binary(g, x[0], len(str(bin(x[0]))[2:]), p) == h)

[3842632, 2902, 147536]
True


In [5]:
#(1)
g = 397628281
h = 404852643
p = 554860571
l = 5044187
# g = 7
# y = 765686981
# p = 35808104999
d = 100
a0, b0 = 1, 0
x = rho_method2(g, h, p, l, a0, b0, d)
print(x)
print(mod_binary(g, x[0], len(str(bin(x[0]))[2:]), p) == h)

[3842632, 2902, 147536]
True


In [95]:
g = 383929062152
h = 329819161243
p = 1023417167557
l = 1740505387

d = 100
a0, b0 = 1, 0
x = rho_method2(g, h, p, l, a0, b0, d)
print(x)
print(mod_binary(g, x[0], len(str(bin(x[0]))[2:]), p) == h)

[1146732679, 41857, 1310792]
True


In [97]:
g = 472974174173350
h = 391980987166049
p = 497264225202533
l = 124316056300633

d = 10000000000
a0, b0 = 1, 0
x = rho_method2(g, h, p, l, a0, b0, d)
print(x)
print(mod_binary(g, x[0], len(str(bin(x[0]))[2:]), p) == h)

[89471828204601, 17146424, 671088720]
True


In [6]:
g = 431147972333069417
h = 511265886395448114
p = 817108219307103587
l = 408554109653551793

d = 10000000000000000
a0, b0 = 1, 0
x = rho_method2(g, h, p, l, a0, b0, d)
print(x)
print(mod_binary(g, x[0], len(str(bin(x[0]))[2:]), p) == h)

In [ ]:
g = 657139733149567012977
h = 903837092395032338397
p = 1077984309859658267861
l = 8693421853706921515

d = 10000000000000000000
a0, b0 = 1, 0
x = rho_method2(g, h, p, l, a0, b0, d)
print(x)
print(mod_binary(g, x[0], len(str(bin(x[0]))[2:]), p) == h)